## Implementation of Parallel MMDE
The following code implements the P-MMDE. It includes the test functions and the scipy nested parallel DE.
First we include the relevant libraries.

In [1]:
# nested DE for global optimization for a minmax objective function
import json
import time
import numpy as np

from scipy.optimize import differential_evolution
genLL = 10
npopLL = 10
genUL = 5
npopUL = 20

Then we define the optimization problems, including their bounds and their upper and lower level objective functions.

In [2]:
# define the bounds on the search
def get_bounds(t):
    r_minL, r_maxL = 0.0, 10.0
    # define the bounds on the LL search
    boundsLL = [(r_minL, r_maxL)] * t
    r_minU, r_maxU = 0.0, 10.0
    # define the bounds on the ul search
    boundsUL = [(r_minU, r_maxU)] * t
    return boundsLL, boundsUL

In [3]:
# objective function
def objectiveUL(x, t):
    resultll = differential_evolution(objectiveLL, bounds=get_bounds(t)[0], maxiter=10, popsize=10, workers=1, args=(x, t))
    return -resultll.fun
# objective function
def objectiveLL(x, y, t):
    f1 = sum((x[t] - 5) ** 2 - (y[t] - 5) ** 2 for t in range(t)) #-f8
    # f2 = 4 * (x[0] - 2) ** 2 - 2 * y[0] ** 2 + x[0] ** 2 * y[0] - y[1] ** 2 + 2 * x[1] ** 2 * y[1]
    # f5 = - (x[0] - 1) * y[0] - (x[1] - 2) * y[1] - (x[2] - 1) * y[2] + 2 * x[0] ** 2 + 3 * x[1] ** 2 + x[2] ** 3 - y[0]**2 - y[1]**2 - y[2]**2
    # f7 = 2 * x[0] * x[4] + 3 * x[3] * x[1] + x[4] * x[2] + 5 * y[3] ** 2 + 5 * y[4] ** 2 - x[3] * (y[3] - y[4] - 5) + x[
    #      4] * (y[3] - y[4] + 3) + sum(x[i] * (y[i] - 1), 2) - sum(y**2)
    #f9_1 = 3 + sum(- 0.2 * x[t] + 0.3 * y[t] for t in range(t))
    #f9_2 = 3 + sum(0.2 * x[t] - 0.1 * y[t] for t in range(t))
    #f9 = min(f9_1, f9_2) %-f9
    #f10 = np.divide(sum(np.sin(x[t]-y[t])for t in range(t)),sum(np.sqrt(x[t]**2+y[t]**2)for t in range(t))) #send it as f10

    #f11 = np.divide(sum(np.cos(np.sqrt(x[t]**2+y[t]**2))for t in range(t)),sum(np.sqrt(x[t]**2+y[t]**2)+10 for t in range(t)))
    return -f1

In [4]:
# f1 objective (equivelant to f8)
# optimal solution: (5,5) optimal obj: 0
def f1UL(x, t):
    resultll = differential_evolution(f1LL, bounds=get_bounds(t)[0], maxiter=genLL, popsize=npopLL, workers=1, args=(x, t))
    return -resultll.fun


def f1LL(x, y, t):
    f1 = sum((x[t] - 5) ** 2 - (y[t] - 5) ** 2 for t in range(t)) 
    return f1

In [5]:
# f2 objective (equivelant to f9)
# optimal solution: (0,0) optimal obj: 3
def f2UL(x, t):
    resultll = differential_evolution(f2LL, bounds=get_bounds(t)[0],  maxiter=genLL, popsize=npopLL, workers=1, args=(x, t),disp=False)
    return -resultll.fun


def f2LL(x, y, t):
    f2_1 = 3 + sum(- 0.2 * x[t] + 0.3 * y[t] for t in range(t))
    f2_2 = 3 + sum(0.2 * x[t] - 0.1 * y[t] for t in range(t))
    f2 = min(f2_1, f2_2)
    return -f2

In [6]:
# f3 objective (equivelant to f10)
# optimal solution: (10,2.1257) optimal obj: 0.097794

def f3UL(x, t):
    resultll = differential_evolution(f3LL, bounds=get_bounds(t)[0],  maxiter=genLL, popsize=npopLL, workers=1, args=(x, t),disp=False)
    return -resultll.fun


def f3LL(x, y, t):
    f3 = np.divide(sum(np.sin(x[t]-y[t])for t in range(t)),sum(np.sqrt(x[t]**2+y[t]**2)for t in range(t))) #send it as f3
    return f3

In [7]:
# f4 objective (equivelant to f11)
# optimal solution: (7.0441,10) optimal obj: 0.042488

def f4UL(x, t):
    resultll = differential_evolution(f4LL, bounds=get_bounds(t)[0], maxiter=genLL, popsize=npopLL, workers=1, args=(x, t),disp=False)
    return -resultll.fun


def f4LL(x, y, t):
    f4 = np.divide(sum(np.cos(np.sqrt(x[t]**2+y[t]**2))for t in range(t)),sum((np.sqrt(x[t]**2+y[t]**2)+10) for t in range(t)))
    #f = np.divide(sum(np.cos(np.sqrt(x[t]**2+y[t]**2))for t in range(t)),sum(np.sqrt(x[t]**2+y[t]**2)+10 for t in range(t))
    return -f4

Then we run the experiments for different number of cores, different dimensionality of the problem and a number of runs. We then save the results in a txt file.

In [ ]:
if __name__ == "__main__":
    tfunction = 1
    for core in [1,2,4,8,16,24]: #removed the 1/add it again
        for t in [1,2,5,10]:
            rt = []
            for i in range(0, 20):
                start = time.time()
                result = differential_evolution(f1UL, bounds=get_bounds(t)[1],  strategy='best1bin',maxiter=genUL, workers=core,
                                                popsize=npopUL, args=(t,), updating='deferred',disp=False)
                                                #updating='deferred', args=(t,))
                end = time.time()
                rt.append((core, end - start, result.x.tolist(), result.fun.tolist()))
                #print('Runtime %s cores:  %s' % (core, end - start))
                #print('Runtime %s result x:  %s' % (core, result.x))
                #print('Runtime %s result f:  %s' % (core, result.fun))
            with open(f'results_fun{tfunction}_dim{t}_cores{core}.txt', 'w') as f:
                f.write(json.dumps(rt))
                

In [8]:
if __name__ == "__main__":
    tfunction = 1
    for core in [1,2,4,8,16,24]: #removed the 1/add it again
        t = 1
        for npopUL in [4,8,16,24,48]:
            rt = []
            for i in range(0, 20):
                start = time.time()
                result = differential_evolution(f1UL, bounds=get_bounds(t)[1],  strategy='best1bin',maxiter=genUL, workers=core,
                                                popsize=npopUL, args=(t,), updating='deferred',disp=False)
                                                #updating='deferred', args=(t,))
                end = time.time()
                rt.append((core, end - start, result.x.tolist(), result.fun.tolist()))
                #print('Runtime %s cores:  %s' % (core, end - start))
                #print('Runtime %s result x:  %s' % (core, result.x))
                #print('Runtime %s result f:  %s' % (core, result.fun))
            with open(f'results_fun{tfunction}_npop{npopUL}_cores{core}.txt', 'w') as f:
                f.write(json.dumps(rt))

In [ ]:
if __name__ == "__main__":
    tfunction = 4
    for core in [1,2,4,8,16,24]: #removed the 1/add it again
        for t in [1,2,5,10]:
            rt = []
            for i in range(0, 20):
                start = time.time()
                result = differential_evolution(f4UL, bounds=get_bounds(t)[1],  strategy='best2bin',maxiter=genUL, workers=core,
                                                popsize=npopUL, args=(t,), updating='deferred',disp=False)
                                                #updating='deferred', args=(t,))
                end = time.time()
                rt.append((core, end - start, result.x.tolist(), result.fun.tolist()))
                #print('Runtime %s cores:  %s' % (core, end - start))
                #print('Runtime %s result x:  %s' % (core, result.x))
                #print('Runtime %s result f:  %s' % (core, result.fun))
            with open(f'results_fun{tfunction}_dim{t}_cores{core}.txt', 'w') as f:
                f.write(json.dumps(rt))

The following two test functions, f5 and f6, are tested only on their original dimensionalities [2,2]. We should add their bounds also here, as they are different than the rest.

In [43]:
# define the bounds on the search
def get_boundsf5():
    r_minL, r_maxL = 0.0, 10.0
    # define the bounds on the LL search
    boundsLL = [(r_minL, r_maxL)] * 2
    boundsUL = ([-0.5, 0.0],[ 0.5, 1.0])
    return boundsLL, boundsUL

In [48]:
# f5 objective (equivelant to f12)
# optimal solution: (0.5,0.25/0,0) optimal obj: 0.25

def f5UL(x):
    resultll = differential_evolution(f5LL, bounds=get_boundsf5()[0],  maxiter=genLL, popsize=npopLL, workers=1, args=(x, ))
    return -resultll.fun


def f5LL(x, y):
    f5 = 100*(x[1]-x[0]**2)**2 + (1 - x[0])**2 - y[0]*(x[0] + x[1]**2) - y[1]*(x[0]**2 + x[1])
    return f5


To run the experiments for f5 we should run the following code: 

In [50]:
if __name__ == "__main__":
    tfunction = 5
    for core in [48]:#[1,2,4,8,16,32,48]:
            rt = []
            for i in range(0, 1):
                start = time.time()
                result = differential_evolution(f5UL, bounds=get_boundsf5()[1],  strategy='randtobest1bin',maxiter=genUL, workers=core,
                                                popsize=npopUL,
                                                updating='deferred',disp=False)
                end = time.time()
                rt.append((core, end - start, result.x.tolist(), result.fun.tolist()))
                print('Runtime %s cores:  %s' % (core, end - start))
                print('Runtime %s result x:  %s' % (core, result.x))
                print('Runtime %s result f:  %s' % (core, result.fun))
            with open(f'run-times_fun{tfunction}_{2}_{core}.txt', 'w') as f:
                f.write(json.dumps(rt))

Runtime 48 cores:  2.070824146270752
Runtime 48 result x:  [-0.48626652  0.59174923]
Runtime 48 result f:  0.21279452103135044


We define the bounds and the objectives for f6

In [11]:
# define the bounds on the search
def get_boundsf6():
    r_minL, r_maxL = 0.0, 10.0
    # define the bounds on the LL search
    boundsLL = [(r_minL, r_maxL)] * 2
    r_minU, r_maxU = -1.0, 3.0
    # define the bounds on the ul search
    boundsUL = [(r_minU, r_maxU)] * 2
    return boundsLL, boundsUL

In [12]:
# f6 objective
# optimal solution: (1,1/any,any) optimal obj: 1

def f6UL(x):
    resultll = differential_evolution(f6LL, bounds=get_boundsf6()[0],  maxiter=genLL, popsize=npopLL, workers=1, args=(x, ),disp=False)
    return -resultll.fun


def f6LL(x, y):
    f6 = (x[0]-2)**2 + (x[1] - 1 )**2 + y[0]*(x[0]**2-x[1]) + y[1]*(x[0]+x[1]-2)
    return f6

To run the experiments for f6 we should run the following code:

In [13]:
if __name__ == "__main__":
    tfunction = 6
    for core in [48]:#[1,2,4,8,16,32,48]:
            rt = []
            for i in range(0, 1):
                start = time.time()
                result = differential_evolution(f6UL, bounds=get_boundsf6()[1],  strategy='randtobest1bin',maxiter=genUL, workers=core,
                                                popsize=npopUL,
                                                updating='deferred',disp=False)
                end = time.time()
                rt.append((core, end - start, result.x.tolist(), result.fun.tolist()))
                print('Runtime %s cores:  %s' % (core, end - start))
                print('Runtime %s result x:  %s' % (core, result.x))
                print('Runtime %s result f:  %s' % (core, result.fun))
            with open(f'run-times_fun{tfunction}_{2}_{core}.txt', 'w') as f:
                f.write(json.dumps(rt))

Runtime 48 cores:  2.4552931785583496
Runtime 48 result x:  [0.66666667 0.66666666]
Runtime 48 result f:  -0.9999999999999999
